In [3]:
import pandas as pd
import re
import os
from dotenv import main
from sqlalchemy import create_engine, text, String, Numeric, Date
from datetime import datetime

In [4]:
main.load_dotenv(dotenv_path="../../Env/.env", override= True)

True

In [5]:
financeiro = create_engine(os.getenv("DB_DIALECT") + "financeiro")
stage = create_engine(os.getenv("DB_DIALECT") + "stage")

In [6]:
caminho_arquivo = "base/Planilha de Fornecedores - 12.06.2024.xlsb"

In [7]:
nome_colunas = ['valor', 'num_doc_ref',	'cod_conta_contabil', 'cod_centro_custo', 'cod_empresa',	'dt_lancamento',	'dt_documento',	'texto',	'atribuicao',	'cod_fornecedor',	'desc_fornecedor',	'dt_referencia',	'tipo',	'ref_nome',	'ref2_nome']

In [8]:
df = pd.read_excel(caminho_arquivo, "BD")

In [9]:
df2 = df

In [29]:
df = df2

In [30]:
# Removendo nulos
df = df[(~df[df.columns[0]].isna()) & (df[df.columns[0]] != "nan")]

In [31]:
# Selecionando apenas as colunas necessárias
df = df[[
    df.columns[0], 
    df.columns[1],
    df.columns[2],
    df.columns[3],
    df.columns[4],
    df.columns[5],
    df.columns[6],
    df.columns[7],
    df.columns[8],
    df.columns[9],
    df.columns[10],
    df.columns[11],
    df.columns[14],
    df.columns[18],
    df.columns[19]
    ]]

In [32]:
df = df.rename(columns = dict(zip(df.columns, nome_colunas)))

In [33]:
df["dt_lancamento"] = pd.to_datetime(df["dt_lancamento"], format='%d.%m.%Y')
df["dt_documento"] = pd.to_datetime(df["dt_documento"], format='%d.%m.%Y')


# Ajustando o tipo de dt_referencia
df["dt_referencia"] = df["dt_referencia"].astype('str')
df.loc[df["dt_referencia"] == "2023.1", "dt_referencia"] = "2023.10"
df.loc[df["dt_referencia"] == "2024.1", "dt_referencia"] = "2024.10"

df["dt_referencia"] = pd.to_datetime(df["dt_referencia"], format='%Y.%m', dayfirst = True)

# Corrigindo tipo
df.loc[df["tipo"] == "ACT 2023", "tipo"] = "act"
df.loc[df["tipo"] == "BGT 2023", "tipo"] = "bgt"
df.loc[df["tipo"] == "ACT 2024", "tipo"] = "act"
df.loc[df["tipo"] == "BGT 2024", "tipo"] = "bgt"

# Corrigindo arredondamento
df["valor"] = df["valor"].round(2)

In [34]:
df = df[df["dt_referencia"] >= "2024-01-01"]

In [35]:
df.to_csv("dados_fornecedores.csv", index = False)

In [27]:
# df = df[["valor", "num_doc_ref", ""]]

In [28]:
# df.to_sql("custo_fornecedores_upsert", stage, index = False, if_exists = "replace")

837